## Automatic Speech Recognition Inference using Online Endpoints

This sample shows how to deploy `automatic-speech-recognition` type models to an online endpoint for inference.

### Task
`automatic-speech-recognition` (ASR) converts a speech signal to text, mapping a sequence of audio inputs to text outputs. Virtual assistants like Siri and Alexa use ASR models to help users everyday, and there are many other useful user-facing applications like live captioning and note-taking during meetings.

### Model
Models that can perform the `automatic-speech-recognition` task are tagged with `task: automatic-speech-recognition`. We will use the `openai-whisper-large` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import-model-from-huggingface.ipynb) and then use them for inference. 

### Inference data
We will use custom audio files that have been uploaded to the cloud. \
You can replace the links with any audio file stored on the cloud and verify inference.
- Most common audio formats (m4a, wav, flac, wma, mp3 etc.) are supported.
- The whisper model can process only 30 seconds of data at a time, so if the file you upload is longer than 30 seconds, only the first 30 seconds will be transcribed. This can be circumvented by splitting the file into 30 second chunks.

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
        credential,
        subscription_id =  "<SUBSCRIPTION_ID>",
        resource_group_name =  "<RESOURCE_GROUP>",
        workspace_name =  "<WORKSPACE_NAME>"
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")


### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `automatic-speech-recognition` task. In this example, we use the `openai-whisper-large` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [ ]:
model_name = "openai-whisper-large"
model_version = "1"
foundation_model=registry_ml_client.models.get(model_name, model_version)
print ("\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(foundation_model.name, foundation_model.version, foundation_model.id))

### 3. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
import time, sys
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, OnlineRequestSettings

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "asr-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for " + foundation_model.name + ", for automatic-speech-recognition task",
    auth_mode="key"
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS4_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 4. Test the endpoint with sample data

We will fetch the sample scoring request and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [ ]:
import pandas as pd

# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response=workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./sample-request/sample_score.json"
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe 
response_df = pd.read_json(response)
response_df.head()

### 5. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()